In [63]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [17]:
MAX_SEQ = 30000
K = 16
N = 16000

In [18]:
strain2num = {'B.1.1.7': 0, 'B.1.427': 1, 'B.1.526': 2, 'P.1': 3}

In [19]:
acid2num = {'A': [1, 0, 0, 0], 'C': [0, 1, 0, 0], 'G':[0, 0, 1, 0], 'T': [0, 0, 0, 1], 
            'N': [0, 0, 0, 0], 'K': [0, 0, 0.5, 0.5], 'Y': [0, 0.5, 0, 0.5] , 'W': [0.5, 0, 0, 0.5], 'R': [0.5, 0, 0.5, 0], 'M':[0.5, 0.5, 0, 0] , 'S': [0, 0.5, 0.5, 0], 'O': [0, 0, 0, 0]}

In [20]:
with open("dataset/Challenge.fa", "r") as f:
    a = f.readlines()
    labels = []

    for i in range(int(N/2)):
        labels.append(strain2num[a[i*2][1:].split("_")[0]])
    
    sequences = []
    for i in range(int(N/2)):
        base = list(a[i*2+1].replace("\n", ""))
        base.extend(["O" for j in range(MAX_SEQ - len(a[i*2+1]))])
        seq = list(map(lambda x: acid2num[x], base))
        sequences.append(seq)
del a

In [21]:
datas = np.array(sequences, dtype="float32")

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D, Conv2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [55]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

X = np.array(datas.reshape(-1, MAX_SEQ-1, 4), dtype="float32")
Y = np.array(labels).reshape(-1, 1)

Y = enc.fit_transform(Y)
Y = np.array(Y.toarray(), dtype="int32")

In [56]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=0)

In [72]:
model = Sequential([
#     Conv1D(16, 11, input_shape=(MAX_SEQ-1, 4), activation="relu"),
#     Conv1D(16, 11, activation="relu"),
#     AveragePooling1D(),
#     Conv1D(32, 15, activation="relu"),
#     Conv1D(32, 15, activation="relu"),
#     AveragePooling1D(),
#     AveragePooling1D(),
#     Conv1D(64, 21, activation="relu"),
    Dense(512, input_shape=(MAX_SEQ-1, 4), activation="relu"),
    GlobalAveragePooling1D(),   
    Dense(512, activation="relu"),
    Dense(128, activation="relu"),
    Dense(4),
    Activation('softmax'),
])

optimizer = Adam(learning_rate=0.005)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [73]:
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=10,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_val, y_val),
)

Epoch 1/10
225/225 [==============================] - 609s 3s/step - loss: 1.3897 - accuracy: 0.2366 - val_loss: 1.3873 - val_accuracy: 0.2587
Epoch 2/10
225/225 [==============================] - 761s 3s/step - loss: 1.3867 - accuracy: 0.2536 - val_loss: 1.3867 - val_accuracy: 0.2450
Epoch 3/10
225/225 [==============================] - 925s 4s/step - loss: 1.3866 - accuracy: 0.2502 - val_loss: 1.3876 - val_accuracy: 0.2250
Epoch 4/10
225/225 [==============================] - 923s 4s/step - loss: 1.3865 - accuracy: 0.2471 - val_loss: 1.3874 - val_accuracy: 0.2250
Epoch 5/10
225/225 [==============================] - 727s 3s/step - loss: 1.3869 - accuracy: 0.2393 - val_loss: 1.3867 - val_accuracy: 0.2587
Epoch 6/10
180/225 [=======================>......] - ETA: 2:00 - loss: 1.3866 - accuracy: 0.2510

KeyboardInterrupt: 

In [ ]:
from tensorflow import keras
keras.backend.clear_session()

In [49]:
y_train.shape, x_train.shape

((7200, 4), (7200, 29999, 4))

In [ ]:
np.fft.fft(x_train)